In [1]:
import os
import pandas as pd
%load_ext memory_profiler

from arboreto.algo import grnboost2, genie3
from arboreto.utils import load_tf_names

In [2]:
ex_matrix = pd.read_csv('~/scratch/net1_expression_data.tsv', sep='\t')

In [3]:
print(ex_matrix.shape)
ex_matrix.head()

(805, 1643)


,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,...,G1634,G1635,G1636,G1637,G1638,G1639,G1640,G1641,G1642,G1643
0,0.425448,0.017829,0.907989,0.448247,0.172324,0.273489,0.843766,0.648201,1.004533,0.365305,...,0.011979,0.963306,1.169870,0.331381,0.350600,0.822844,0.304483,0.319917,0.364280,0.765945
1,0.442400,0.050525,0.869368,0.445851,0.173311,0.274889,0.764049,0.747870,1.022589,0.434106,...,0.022247,1.014137,0.888465,0.281649,0.485940,0.915617,0.317507,0.238074,0.509130,0.691403
2,1.056847,0.208454,0.467448,0.505077,0.244883,0.208451,0.665355,1.192092,0.824068,0.146987,...,0.422066,0.895203,1.028826,0.825126,0.444819,0.349069,0.042310,0.165208,0.952178,0.678781
3,1.117226,0.003001,0.317654,0.387204,0.253792,0.179360,0.939244,0.868668,0.963028,0.233785,...,0.001163,1.046540,1.058098,0.484225,0.150689,0.449126,0.125197,0.000047,0.878127,0.566691
4,0.971068,0.001056,0.354651,0.474532,0.207718,0.102833,0.745871,0.909753,1.151865,0.318988,...,0.000845,1.041745,1.061129,0.384363,0.326859,0.512270,0.261410,0.000156,0.883981,0.646715


In [4]:
tf_names = pd.read_csv('~/scratch/net1_transcription_factors.tsv', sep='\t', header=None).values.flatten().tolist()

In [5]:
len(tf_names)

195

In [6]:
from dask_jobqueue import SLURMCluster
from distributed import Client

In [7]:
cluster = SLURMCluster(
    queue='batch',
    cores=31,
    processes=31,
    memory="150 GB"
)

print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p batch
#SBATCH -n 1
#SBATCH --cpus-per-task=31
#SBATCH --mem=140G
#SBATCH -t 00:30:00

JOB_ID=${SLURM_JOB_ID%;*}

/gpfs_home/spate116/ml/bin/python3.7 -m distributed.cli.dask_worker tcp://172.20.208.2:44148 --nthreads 1 --nprocs 31 --memory-limit 4.84GB --name name --nanny --death-timeout 60



In [8]:
cluster.scale(jobs=1)

from dask.distributed import Client
client = Client(cluster)

In [10]:
client

Client Scheduler: tcp://172.20.208.2:44148 Dashboard: http://172.20.208.2:8787/status,Cluster Workers: 25 Cores: 25 Memory: 121.00 GB


In [11]:
%%time
%%memit
network = grnboost2(expression_data=ex_matrix.to_numpy(),
                    gene_names=ex_matrix.columns,
                    client_or_address=client)

peak memory: 540.16 MiB, increment: 364.23 MiB
CPU times: user 42 s, sys: 2.12 s, total: 44.1 s
Wall time: 12min 30s


In [12]:
network.head()

,TF,target,importance
108,G109,G1406,154.637628
187,G188,G938,145.545418
1404,G1406,G109,141.415057
48,G49,G978,128.764061
15,G16,G687,122.192907


In [19]:
network.to_csv('~/data/spate116/GCN/arb_ex.tsv', sep='\t', header=True, index=False)

In [14]:
client.close()
cluster.close()

In [21]:
load_check = pd.read_csv('~/data/spate116/GCN/arb_ex.tsv', sep='\t')

In [22]:
load_check.head()

,TF,target,importance
0,G109,G1406,154.637628
1,G188,G938,145.545418
2,G1406,G109,141.415057
3,G49,G978,128.764061
4,G16,G687,122.192907
